# day8

> Day : Treetop Tree House


...

In [ ]:
#| default_exp day8
NAME = "day8"

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

---

# Example

Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

In [ ]:
#| exports
import numpy as np

example = """
30373
25512
65332
33549
35390
""".strip().split()
ex_heights = np.array([[int(x) for x in row] for row in example])
print(ex_heights)

[[3 0 3 7 3]
 [2 5 5 1 2]
 [6 5 3 3 2]
 [3 3 5 4 9]
 [3 5 3 9 0]]


A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

In [ ]:
#| exports
#| code-fold: true

def edges(heights: np.array) -> np.array:
    """Return Boolean grid with only edges == 1"""
    x, y = heights.shape
    return np.pad(np.zeros((x-2, y-2), dtype=bool),
                  pad_width=1,
                  constant_values=1)

def vismap(heights: np.array) -> np.array:
    """Boolean map: is tree is tallest in some direction?"""
    rows, cols = heights.shape
    ans = edges(heights)
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            height = heights[i,j]
            row, col = heights[i,:], heights[:,j]
            ans[i,j] = ((row[:j] < height).all() or
                        (col[:i] < height).all() or
                        (row[j+1:] < height).all() or
                        (col[i+1:] < height).all())
    return ans

In [ ]:
#| test
assert (edges(ex_heights) == np.array(
[[1,1,1,1,1],
 [1,0,0,0,1],
 [1,0,0,0,1],
 [1,0,0,0,1],
 [1,1,1,1,1]], dtype=bool)
       ).all()

In [ ]:
#| test
assert (vismap(ex_heights) == np.array(
[[1,1,1,1,1],
 [1,1,1,0,1],
 [1,1,0,1,1],
 [1,0,1,0,1],
 [1,1,1,1,1]], dtype=bool)
       ).all()

# Part 1

## Get the data


In [ ]:
with open(f"data/{NAME}_input.txt") as f:
    data = f.read()
data = data.strip().split("\n")
data = np.array([[int(x) for x in row] for row in data])

In [ ]:
vismap(data)

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True, False,  True, ..., False,  True,  True],
       [ True,  True, False, ...,  True, False,  True],
       ...,
       [ True, False,  True, ..., False,  True,  True],
       [ True, False,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

## Solve

In [ ]:
np.sum(vismap(data))

1825

# Part 2

In the example above, consider the middle 5 in the second row:
```
30373
25512
65332
33549
35390
```
A tree's **scenic score** is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

## Calc & Test `scenic_scores()`

In [ ]:
#| exports
#| code-fold: true

def scenic_scores(heights: np.array) -> np.array:
    """Get scenic score of all trees from height map."""
    rows, cols = heights.shape
    ans = np.ones_like(heights)
    for i in range(rows):
        for j in range(rows):
            height = heights[i,j]
            row, col = heights[i,:], heights[:,j]
            #print(row, f"({height})")
            #print(where_if(row[:j] >= height))
            left = j - where_if(row[:j] >= height)[-1]
            up = i - where_if(col[:i] >= height)[-1]
            right = 1 + where_if(row[j+1:] >= height, cols-j-2)[0]
            down = 1 + where_if(col[i+1:] >= height, rows-i-2)[0]
            #print(f"({i},{j}):", left, up, right, down)
            
            ans[i,j] = left * up * right * down
    return ans

In [ ]:
#| exports
#| code-fold: True

def where_if(x, _else=0): 
    """Returns where(x) or [_else] """
    _ = np.where(x)[0]
    return _ if len(_) != 0 else _else * np.ones(1) 

def assert_all(mybool: np.array) -> None:
    """Wraps assert (foo > bar).all()"""
    assert mybool.all()

In [ ]:
#| test
arr = np.array

assert_all(where_if(arr([0,1,1,0])) == arr([1,2]))
assert_all(where_if(arr([0,1,1,0]) > 0) == arr([1,2]))

assert_all(where_if(arr([0,0,0]) > 0) == arr([0]))
assert_all(where_if(arr([0,0,0]) > 0, _else=1) == arr([1]))

In [ ]:
print(scenic_scores(ex_heights))
assert scenic_scores(ex_heights)[1,2] == 4
assert (scenic_scores(ex_heights) == np.array(
[[0,0,0,0,0],
 [0,1,4,1,0],
 [0,6,1,2,0],
 [0,1,8,3,0],
 [0,0,0,0,0]])
).all()

[[0 0 0 0 0]
 [0 1 4 1 0]
 [0 6 1 2 0]
 [0 1 8 3 0]
 [0 0 0 0 0]]


## Solve Part 2

In [ ]:
scenic_scores(data)

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  1, 14, ...,  1,  8,  0],
       [ 0, 12, 12, ...,  8,  1,  0],
       ...,
       [ 0,  1,  2, ...,  1,  8,  0],
       [ 0,  3,  8, ...,  8,  1,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [ ]:
np.max(scenic_scores(data))

235200

----
Footer: nbdev magic

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()